In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.docstore.document import Document
import pandas as pd

#### INDEXING ####

# Load Documents
df = pd.read_csv('/content/amazon_data.csv')

docs = [Document(page_content=row.to_string()) for _, row in df.iterrows()]
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke(""" Expertise and personalization based on the context provided, recommend tailored strategies for saving, investing, and budgeting,
should trends and offer insights into spending habits""")

"Based on the provided stock data, a tailored strategy for saving, investing, and budgeting could involve monitoring trends in the stock's performance to make informed decisions. Insights into spending habits can be gained by analyzing the percent change in the stock's value over time. It is important to consider the volatility and patterns in the stock's movement to develop a personalized financial plan."

In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import load_prompt
from langchain.docstore.document import Document
#### INDEXING ####

# Load CSV file
df = pd.read_csv('/content/amazon_data.csv')

# Convert DataFrame to documents
docs = [Document(page_content=row.to_string()) for _, row in df.iterrows()]

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Load Prompt
prompt =  vectorstore.as_retriever()

prompt_text = """
Expertise and personalization based on the context provided, recommend tailored strategies for saving, investing, and budgeting,
should trends and offer insights into spending habits.
"""

def prompt_fn(inputs):
    return prompt_text

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )
formatted_docs = retriever | format_docs
prompted = formatted_docs | (lambda inputs: {"prompt": prompt_fn(inputs), "question": inputs})
llm_input = prompted | (lambda inputs: inputs["prompt"])
rag_chain = llm_input | llm | StrOutputParser()

# Question
response = rag_chain.invoke("setting long-term financial goals and planning for future milestones like buying a house or traveling.")
print(response)


Based on the context provided, here are some tailored strategies for saving, investing, and budgeting:

1. Saving:
- Set specific savings goals based on your financial situation and priorities.
- Automate your savings by setting up regular transfers to a separate savings account.
- Cut back on unnecessary expenses and find ways to save on everyday purchases.
- Consider opening a high-yield savings account to earn more interest on your savings.

2. Investing:
- Diversify your investment portfolio to spread risk and maximize returns.
- Consider investing in low-cost index funds or ETFs for long-term growth.
- Regularly review and adjust your investment strategy based on your financial goals and risk tolerance.
- Consult with a financial advisor to get personalized investment advice tailored to your needs.

3. Budgeting:
- Track your expenses and create a budget based on your income and spending habits.
- Identify areas where you can cut back on expenses and allocate more funds towards sa

In [ ]:
# Documents
question = "Give answer accordingly that data file provided"
document = "/content/amazon_data.csv"

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

7

In [ ]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.7608413337517983


In [ ]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
df1 = pd.read_csv('/content/amazon_data.csv')

In [ ]:
docs = [Document(page_content=row.to_string()) for _, row in df1.iterrows()]
# Split
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)

In [ ]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
docs = retriever.get_relevant_documents("/content/amazon_data.csv")

In [ ]:
len(docs)

1

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """How to gain excellent profit using Amazon stocks?
{context}

Question: {If i buy 3 shares of Amazon stocks. How much dividend recived after the year?}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['If i buy 3 shares of Amazon stocks. How much dividend recived after the year?', 'context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['If i buy 3 shares of Amazon stocks. How much dividend recived after the year?', 'context'], template='How to gain excellent profit using Amazon stocks?\n{context}\n\nQuestion: {If i buy 3 shares of Amazon stocks. How much dividend recived after the year?}\n'))])

In [ ]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
input_data = {
    "context": "Provide context here if not using retriever directly.",
    "question": "How much Amazon average dividend is received after a year?"
}

In [ ]:
response = rag_chain.invoke(input_data)
print(response)

TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
import pandas as pd

#### INDEXING ####

# Load Documents
df3 = pd.read_csv('/content/amazon_data.csv')
documents = [Document(page_content=row.to_string()) for _, row in df3.iterrows()]

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Define prompt template
template = """How to gain excellent profit using Amazon stocks?
{context}

Question: {How much dividend receive after a year if I buy 5 shares of Amazon.}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
context = format_docs(retriever)  # Ensure you have a proper context string
question = "How much Amazon average dividend is received after a year?"

# Create input string for invocation
input_string = f"Answer the question based only on the following context:\n{context}\n\nQuestion: {question}"

# Run
response = rag_chain.invoke(input_string)
print(response)


AttributeError: 'tuple' object has no attribute 'page_content'

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response
from .models import Item
from .serializers import ItemSerializer

@api_view(['GET'])
def getData(request):
  items = Item.objects.all()
  serializer = ItemSerializer(items, many=True)
  return Response(serializer.data)

In [ ]:
class Car:
  def __init__(self, make, model, year):
    self.make = make
    self.model = model
    self.year = year

class ElectricCar(Car):
 """Represent aspects of a car, specific to electric vehicles."""
 def __init__(self, make, model, year):
  self.make = make
  self.model = model
  self.year = year
  super().__init__(make, model, year)
  self.battery_size = 40
 def get_descriptive_name(self):
  long_name = f"{self.year} {self.make} {self.model}"
  return long_name.title()
 def describe_battery(self):
  print(f"This car has a {self.battery_size}-kWh battery.")
my_leaf = ElectricCar('nissan', 'leaf', 2024)
print(my_leaf.get_descriptive_name())
my_leaf.describe_battery()

2024 Nissan Leaf
This car has a 40-kWh battery.
